# Equal-Weight S&P 500 Index Fund

## Introduction & Library Imports

The S&P 500 is the world's most popular stock market index. The largest fund that is benchmarked to this index is the SPDR® S&P 500® ETF Trust. It has more than US$250 billion of assets under management.

The goal of this section of the course is to create a Python script that will accept the value of your portfolio and tell you how many shares of each S&P 500 constituent you should purchase to get an equal-weight version of the index fund.

## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

Video: https://www.youtube.com/watch?v=xfzGZB4HhEE&list=PLDyxV_SxnHQjg2Tc1Vdk25zraOQVwMD8w&index=30&t=1207s&ab_channel=freeCodeCamp.org

In [1]:
import numpy as np
import pandas as pd
import requests,xlsxwriter,math,json

## Importing Our List of Stocks

The next thing we need to do is import the constituents of the S&P 500.

These constituents change over time, so in an ideal world you would connect directly to the index provider (Standard & Poor's) and pull their real-time constituents on a regular basis.

Paying for access to the index provider's API is outside of the scope of this course. 

There's a static version of the S&P 500 constituents available here. [Click this link to download them now](https://drive.google.com/file/d/1ZJSpbY69DVckVZlO9cC6KkgfSufybcHN/view?usp=sharing). Move this file into the `starter-files` folder so it can be accessed by other files in that directory.

Now it's time to import these stocks to our Jupyter Notebook file.

In [2]:
stocks = pd.read_csv('sp_500_stocks.csv')
stocks

,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV
...,...
500,YUM
501,ZBH
502,ZBRA
503,ZION


## Acquiring an API Token

Now it's time to import our IEX Cloud API token. This is the data provider that we will be using throughout this course.

API tokens (and other sensitive information) should be stored in a `secrets.py` file that doesn't get pushed to your local Git repository. We'll be using a sandbox API token in this course, which means that the data we'll use is randomly-generated and (more importantly) has no cost associated with it.

[Click here](http://nickmccullum.com/algorithmic-trading-python/secrets.py) to download your `secrets.py` file. Move the file into the same directory as this Jupyter Notebook before proceeding.

In [33]:
from secrets import IEX_CLOUD_API_TOKEN

'Tpk_059b97af715d417d9f49f50b51b1c448'

## Making Our First API Call

Now it's time to structure our API calls to IEX cloud. 

We need the following information from the API:

* Market capitalization for each stock
* Price of each stock



In [4]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote/?token={IEX_CLOUD_API_TOKEN}' #f string (sth like format)
data = requests.get(api_url).json()
data

{'symbol': 'AAPL',
 'companyName': 'Apple Inc',
 'primaryExchange': 'K ALECNQSNR ALASAE(OTG/TD SBEM)GL',
 'calculationPrice': 'close',
 'open': 124.89,
 'openTime': 1642734039737,
 'openSource': 'iicoaffl',
 'close': 122.82,
 'closeTime': 1667257167392,
 'closeSource': 'oafcilfi',
 'high': 126.98,
 'highTime': 1641656107939,
 'highSource': '1im p lrdcei eatu5ednye',
 'low': 124.58,
 'lowTime': 1651621111994,
 'lowSource': 'eei cilydteu ndp ra51em',
 'latestPrice': 123.08,
 'latestSource': 'Close',
 'latestTime': 'December 11, 2020',
 'latestUpdate': 1651932954160,
 'latestVolume': 90344193,
 'iexRealtimePrice': None,
 'iexRealtimeSize': None,
 'iexLastUpdated': None,
 'delayedPrice': 127.06,
 'delayedPriceTime': 1611868608533,
 'oddLotDelayedPrice': 123.329,
 'oddLotDelayedPriceTime': 1685297982373,
 'extendedPrice': 123.69,
 'extendedChange': -0.14,
 'extendedChangePercent': -0.00116,
 'extendedPriceTime': 1679771599300,
 'previousClose': 129.37,
 'previousVolume': 85011903,
 'change'

## Parsing Our API Call

The API call that we executed in the last code block contains all of the information required to build our equal-weight S&P 500 strategy. 

With that said, the data isn't in a proper format yet. We need to parse it first.

In [5]:
price = data['latestPrice'] #Not accurate actually
market_cap = data['marketCap']#Not accurate actually 

## Adding Our Stocks Data to a Pandas DataFrame

The next thing we need to do is add our stock's price and market capitalization to a pandas DataFrame. Think of a DataFrame like the Python version of a spreadsheet. It stores tabular data.

In [10]:
my_columns = ['Ticker','Stock Price', 'Market Capitaliztion', 'Number of shares to Buy']
final_df = pd.DataFrame(columns = my_columns)

2141491922697

In [14]:
final_df = final_df.append(pd.Series([symbol,price,market_cap,'N/A'],index = my_columns),ignore_index=True) #Series in pandas is 1D-array, such as column / row
final_df

,Ticker,Stock Price,Market Capitaliztion,Number of shares to Buy
0,AAPL,123.08,2141491922697,N/A


## Looping Through The Tickers in Our List of Stocks

Using the same logic that we outlined above, we can pull data for all S&P 500 stocks and store their data in the DataFrame using a `for` loop.

In [19]:
#Takes very long to run
#final_df = pd.DataFrame(columns = my_columns)
#for stock in stocks['Ticker']:
#    api_url = f'https://sandbox.iexapis.com/stable/stock/{stock}/quote/?token={IEX_CLOUD_API_TOKEN}' #f string (sth like format)
#    data = requests.get(api_url).json()
#    final_df = final_df.append(pd.Series([stock,data['latestPrice'],data['marketCap'],'N/A'],index = my_columns),ignore_index = True)

In [20]:
#final_df

,Ticker,Stock Price,Market Capitaliztion,Number of shares to Buy
0,A,120.32,36934517313,N/A
1,AAL,17.66,10458027609,N/A
2,AAP,159.04,10883630876,N/A
3,AAPL,126.42,2112354496124,N/A
4,ABBV,109.98,188528024711,N/A
...,...,...,...,...
500,YUM,108.89,32108529006,N/A
501,ZBH,147.37,29954536679,N/A
502,ZBRA,389.10,21064904837,N/A
503,ZION,43.56,6947034071,N/A


## Using Batch API Calls to Improve Performance

Batch API calls are one of the easiest ways to improve the performance of your code.

This is because HTTP requests are typically one of the slowest components of a script.

Also, API providers will often give you discounted rates for using batch API calls since they are easier for the API provider to respond to.

IEX Cloud limits their batch API calls to 100 tickers per request. Still, this reduces the number of API calls we'll make in this section from 500 to 5 - huge improvement! In this section, we'll split our list of stocks into groups of 100 and then make a batch API call for each group.

In [25]:
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

In [43]:
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','. join(symbol_groups[i]))
#    print(symbol_strings[i])
final_df = pd.DataFrame(columns = my_columns)
for symbol_strings in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_strings}&types=quote&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_strings.split(','):
       final_df = final_df.append(pd.Series([symbol,data[symbol]['quote']['latestPrice'],data[symbol]['quote']['marketCap'],'N/A'],index = my_columns),ignore_index = True)
final_df  

,Ticker,Stock Price,Market Capitaliztion,Number of shares to Buy
0,A,123.50,38224950781,N/A
1,AAL,17.74,10785059598,N/A
2,AAP,165.36,11173824284,N/A
3,AAPL,128.27,2145539264659,N/A
4,ABBV,110.48,195076807310,N/A
...,...,...,...,...
500,YUM,106.93,33317001153,N/A
501,ZBH,144.45,30538283751,N/A
502,ZBRA,384.13,21125023767,N/A
503,ZION,42.81,7020064362,N/A


## Calculating the Number of Shares to Buy

As you can see in the DataFrame above, we stil haven't calculated the number of shares of each stock to buy.

We'll do that next.

In [64]:
profolio_size = input('Enter the value of your portfolio: ')

try:
    val = float(profolio_size)
except ValueError:
    print('Please enter an integer')
    profolio_size = input('Enter the value of your portfolio: ')
    val = float(protolio_size)

Enter the value of your portfolio: 10000000


In [65]:
position_size = val/len(final_df.index)
for i in range(0,len(final_df.index)):
    final_df.loc[i, 'Number of shares to Buy'] = math.floor(position_size/final_df.loc[i, 'Stock Price']) #math.floor can round down values
final_df

,Ticker,Stock Price,Market Capitaliztion,Number of shares to Buy
0,A,123.50,38224950781,160
1,AAL,17.74,10785059598,1116
2,AAP,165.36,11173824284,119
3,AAPL,128.27,2145539264659,154
4,ABBV,110.48,195076807310,179
...,...,...,...,...
500,YUM,106.93,33317001153,185
501,ZBH,144.45,30538283751,137
502,ZBRA,384.13,21125023767,51
503,ZION,42.81,7020064362,462


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

### Initializing our XlsxWriter Object

In [100]:
writer = pd.ExcelWriter('recommended trades.xlsx', engine = 'xlsxwriter')
final_df.to_excel(writer, 'Recommended Trades', index=False)

### Creating the Formats We'll Need For Our `.xlsx` File

Formats include colors, fonts, and also symbols like `%` and `$`. We'll need four main formats for our Excel document:
* String format for tickers
* \\$XX.XX format for stock prices
* \\$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

In [101]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_format = writer.book.add_format(
{
    'font_color': background_color,
    'bg_color': font_color,
    'border': 1
})

dollar_format = writer.book.add_format(
{
    'num_format': '$0.00',
    'font_color': background_color,
    'bg_color': font_color,
    'border': 1
})

integer_format = writer.book.add_format(
{
    'num_format': '$0',
    'font_color': background_color,
    'bg_color': font_color,
    'border': 1
})

### Applying the Formats to the Columns of Our `.xlsx` File

We can use the `set_column` method applied to the `writer.sheets['Recommended Trades']` object to apply formats to specific columns of our spreadsheets.

Here's an example:

```python
writer.sheets['Recommended Trades'].set_column('B:B', #This tells the method to apply the format to column B
                     18, #This tells the method to apply a column width of 18 pixels
                     string_template #This applies the format 'string_template' to the column
                    )
```

In [102]:
#writer.sheets['Recommended Trades'].set_column('A:A',18, string_format)
#writer.sheets['Recommended Trades'].set_column('B:B',18, string_format)
#writer.sheets['Recommended Trades'].set_column('C:C',18, string_format)
#writer.sheets['Recommended Trades'].set_column('D:D',18, string_format)
#writer.save()

writer.sheets['Recommended Trades'].write('A1','Ticker',string_format)
writer.sheets['Recommended Trades'].write('B1','Stock Price',dollar_format)
writer.sheets['Recommended Trades'].write('C1','Market Capitaliztion',dollar_format)
writer.sheets['Recommended Trades'].write('D1','Number of shares to Buy',integer_format)

0

This code works, but it violates the software principle of "Don't Repeat Yourself". 

Let's simplify this by putting it in 2 loops:

In [104]:
column_formats = {
    'A': ['Ticker',string_format],
    'B': ['Stock Price',dollar_format],
    'C': ['Market Capitaliztion',dollar_format],
    'D': ['Number of shares to Buy',integer_format],
}

for column in column_formats.keys():
    writer.sheets['Recommended Trades'].set_column(f'{column}:{column}',18, column_formats[column][1])
    writer.sheets['Recommended Trades'].write(f'{column}1',column_formats[column][0],column_formats[column][1])


## Saving Our Excel Output

Saving our Excel file is very easy:

In [105]:
writer.save()